In [ ]:
pip install opencv


In [ ]:
pip install mediapipe

In [ ]:
pip install times


In [1]:
import mediapipe as mp
import time
import cv2

In [2]:
import cv2
import mediapipe as mp

class handDetector:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5, model_complexity=1):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.model_complexity = model_complexity
        self.mp_hands = mp.solutions.hands.Hands(
            self.mode, self.maxHands,
            model_complexity=self.model_complexity,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.trackCon
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.results = None

    def findHands(self, image, draw=True):
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.mp_hands.process(imageRGB)
        if self.results.multi_hand_landmarks and draw:
            for handLms in self.results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(image, handLms, mp.solutions.hands.HAND_CONNECTIONS)
        return image

    def findPos(self, image, handNumber=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNumber]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(image, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        return lmList


In [ ]:
def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    while True:
        success, image = cap.read()
        image = detector.findHands(image)
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        image = cv2.flip(image, 1)
        cv2.putText(
            image,
            str(int(fps)),
            (500,400),
        cv2.FONT_HERSHEY_PLAIN,
        3,
        (120, 52, 40),
        3,
        )
        cv2.imshow("Camera result", image)
        landmarks = detector.findPos(image)
        if len(landmarks) != 0:
            print(landmarks[4])
        cv2.waitKey(1)
        
if __name__ == "__main__":
    main()